<a href="https://colab.research.google.com/github/tomonari-masada/course2024-sml/blob/main/11_document_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# クラスタリング
* クラスタリングの代表的な手法であるk平均法を使ってみる。

## 例題: テキスト・クラスタリング

* Transformesベースの日本語対応言語モデルを使って、テキストのベクトル表現を得る。
  * テキストをベクトルとして表現することを「embedする」と言う。
* そして、テキストのembeddingをk平均法でクラスタリングする。

* ランタイムのタイプをGPUにしておく。

## インストール

### spaCyの日本語モデル

In [ ]:
!python -m spacy download ja_core_news_sm

### Hugging Faceのdatasetsライブラリ

In [ ]:
!pip install datasets

### SentenceTransformersライブラリ
* テキストの埋め込みを得るために便利なライブラリ。
  * https://sbert.net/index.html

In [ ]:
!pip install -U sentence-transformers

* 日本語対応BERTを使うためのインストール。

In [ ]:
!pip install fugashi ipadic

## インポート

In [ ]:
from tqdm.auto import tqdm
import collections
import numpy as np

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import spacy

from datasets import load_dataset
from sentence_transformers import SentenceTransformer

## データセット
* livedoorニュースコーパスを使う。

In [ ]:
dataset = load_dataset("llm-book/livedoor-news-corpus")

In [ ]:
collections.Counter(dataset["train"]["category"])

In [ ]:
dataset["train"]["title"][0]

In [ ]:
dataset["train"]["content"][0]

## 日本語BERTによる埋め込み

* 東北大学が提供している、日本語に対応したBERTを使う。
  * https://huggingface.co/tohoku-nlp/bert-base-japanese-char-whole-word-masking

* SentenceTransformerでの埋め込みについては、下のWebページを参照。
  * https://sbert.net/examples/applications/computing-embeddings/README.html

In [ ]:
model = SentenceTransformer("tohoku-nlp/bert-base-japanese-char-whole-word-masking")

* 全タイトルを埋め込む。
  * RTX3080搭載PCを使うと5秒以内で終わる。

In [ ]:
embeddings = model.encode(dataset["train"]["title"], show_progress_bar=True)

* 全記事内容を埋め込む。  
  * RTX3080搭載PCを使うと1分で終わる。

In [ ]:
#content_embeddings = model.encode(dataset["train"]["content"], show_progress_bar=True)

* 埋め込みを保存。

In [ ]:
with open('bert_embeddings.npy', 'wb') as f:
  np.save(f, embeddings)

In [ ]:
#with open('bert_content_embeddings.npy', 'wb') as f:
#  np.save(f, content_embeddings)

* 読み込みは以下のようにする。

In [ ]:
with open('bert_embeddings.npy', 'rb') as f:
  embeddings = np.load(f)

In [ ]:
#with open('bert_content_embeddings.npy', 'rb') as f:
#  content_embeddings = np.load(f)

## クラスタのラベリングに使う単語の抽出
* ラベルとして使う単語を形態素解析によって抽出する。

In [ ]:
nlp = spacy.load("ja_core_news_sm")
corpus = []
for text in tqdm(dataset["train"]["title"]):
  corpus.append(" ".join([token.lemma_ for token in nlp(text)]))

* scikit-learnでTF-IDFを計算する。
* `TfidfVectorizer`の`min_df`パラメータは適当に調節する。
  * クラスタのラベリングに向かないマイナーな単語が含まれないようにする。

In [ ]:
vectorizer = TfidfVectorizer(min_df=10)
X_train = vectorizer.fit_transform(corpus).toarray()
vocab = np.array(vectorizer.get_feature_names_out())

In [ ]:
vocab.size

In [ ]:
print(list(vocab))

## ラベリング用単語の埋め込み

* その単語を含むテキストの埋め込みベクトルの加重平均を求める。
  * 重みはTF-IDFの値を元に定める。

In [ ]:
text_weights = X_train / X_train.sum(0)

In [ ]:
vocab_embeddings = np.dot(text_weights.T, embeddings)

## 単語埋め込みのクラスタリング



### k-平均法によるクラスタリング

In [ ]:
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=123)
kmeans.fit(embeddings)
centers = kmeans.cluster_centers_

* クラスタの重心を保存。

In [ ]:
with open(f'bert_centers_{n_clusters}.npy', 'wb') as f:
  np.save(f, centers)

In [ ]:
with open(f'bert_centers_{n_clusters}.npy', 'rb') as f:
  centers = np.load(f)

### クラスタのサイズを調べる

* クラスタのインデックスをキーとし、そのサイズを値とする辞書を作る。

In [ ]:
unique, counts = np.unique(kmeans.labels_, return_counts=True)
size_dict = dict(zip(unique, counts))

* 辞書のエントリを、キーではなく値でソートする。

In [ ]:
sorted_clusters = [k for k, v in sorted(size_dict.items(), key=lambda item: item[1], reverse=True)]

In [ ]:
counts[sorted_clusters]

In [ ]:
print(sorted_clusters)

## クラスタのラベリング
* 各クラスタの重心に近い20単語でラベリングする。

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(vocab_embeddings, centers)

In [ ]:
for i in range(similarities.shape[-1]):
  indices = np.argsort(- similarities[:,i])
  print(vocab[indices[:20]])